# COE 321K - Homework 3: Computational Truss Analysis
Here, we go thru a detailed process of deriving a Python script that will calculate the 
- nodal displacements
- bar forces,
- external nodal forces

for an arbitrary two- or three-dimensional truss structure.

We will eventually combine all the code into a single source code file.

## Pre-processing: Truss Code Inputs & Data Structures, Stiffness & Force Assembly.

**General Definitions**
- **`num_dims`**: Number of spatial dimensions  
- **`num_nodes`**: Number of nodes  
- **`node(i, j)`**: $x_j$ position of the $i^{\text{th}}$ node (the $j^{\text{th}}$ spatial coordinate of the $i^{\text{th}}$ node)  
- **`dof_per_node`**: Number of degrees of freedom (DOFs) per node  
- **`gcon(i, j)`**: Global degree of freedom number of node $i$, local DOF $j$  
- **`total_num_dofs`**: Total number of DOFs in the structure  

**Element-Specific Definitions**
- **`num_elements`**: Number of elements  
- **`element_nodes(i, j)`**: Global node number of the $j^{\text{th}}$ local node of element $i$  
- **`E(i)`**: Young's modulus for element $i$ (in Pascals, $Pa$)  
- **`A(i)`**: Cross-sectional area of element $i$ (in square meters, $m^2$)  
- **`L(i)`**: Length of element $i$ (in meters, $m$)  
- **`element_cosines(i, j)`**: Direction cosine between element $i$ from local node 1 to local node 2, relative to the $x_j$-axis  

**Boundary Conditions (BCs)**
- **`num_force_BCs`**: Number of non-zero force boundary conditions  

**Force Boundary Conditions**
- **`force_node(i)`**: Node number for force BC $i$  
- **`force_dof(i)`**: Local DOF number for force BC $i$  
- **`force_value(i)`**: Force value for force BC $i$ (in Newtons, $N$)  

**Displacement Boundary Conditions**
- **`disp_node(i)`**: Node number for the $i^{\text{th}}$ displacement BC  
- **`disp_dof(i)`**: Node DOF number for the $i^{\text{th}}$ displacement BC  
- **`disp_value(i)`**: Node displacement value for the $i^{\text{th}}$ displacement BC (in meters, $m$)  
---

In [50]:
import numpy as np
import pandas as pd
from pprint import pprint
from typing import List, Dict, Tuple

### "Nodes" File
We will build a function that reads the nodes file and then retur 1-based-indexed nodes matrix, number of nodes, and number of dimensions.

In [85]:
def read_nodes_file(filepath: str = 'inputs/nodes') -> Tuple[np.ndarray, int, int]:
   # Read the first two lines manually
    with open(filepath, 'r') as file:
        num_dims = int(file.readline().strip())
        num_nodes = int(file.readline().strip())
    # Read the rest of the file (starting from the third line)
    node_file = pd.read_csv(filepath, sep=r'\s+', header=None, skiprows=2)
    # Convert to numpy array
    nodes = node_file.to_numpy()
    # Add an extra row to accommodate 1-based indexing
    padded_nodes = np.pad(nodes, ((1,0), (0,0)), 'constant', constant_values=np.nan)

    return padded_nodes, num_nodes, num_dims

# Call the function to get node mx, num_nodes, and num_dims
node, num_nodes, num_dims = read_nodes_file(filepath='test_cases_2d/nodes')

# compute dof per node
dof_per_node = num_dims # for Truss only
# total number of DOFs in a structure
total_num_dofs = (len(node) - 1) * dof_per_node

print(f"Number of Nodes: {num_nodes}\nNumber of dims: {num_dims}\ndof per node:{dof_per_node}\ntotal dofs:{total_num_dofs}")
print("\nnode matrix:")
pprint(node)

Number of Nodes: 4
Number of dims: 2
dof per node:2
total dofs:8

node matrix:
array([[nan, nan, nan],
       [ 1.,  0.,  0.],
       [ 2.,  1.,  0.],
       [ 3.,  0.,  1.],
       [ 4.,  1.,  1.]])


Now we will go ahead and build our **$gcon(i, j)$**

#### Build $gcon(i,j)$

In [52]:
def build_global_connectivity(node_mx: np.ndarray, dof_per_node) -> np.ndarray:
    global_connectivity = np.zeros_like(node_mx, dtype=int)
    # form gcon data structure
    for node_num in range(1, len(node_mx)): # iterate thru all nodes
        for local_dof in range(1, dof_per_node + 1): # iterate thru all dof components
            # assign global DOF to each element:
            global_connectivity[node_num, local_dof] = dof_per_node * (node_num - 1) + local_dof
    
    return global_connectivity

global_connectivity = build_global_connectivity(node, dof_per_node)

print("gcon matrix:")
pprint(global_connectivity)


gcon matrix:
array([[0, 0, 0],
       [0, 1, 2],
       [0, 3, 4],
       [0, 5, 6],
       [0, 7, 8]])


### "Element"file

In [86]:
def read_elements_file(filepath: str = 'inputs/elements') -> Tuple[np.ndarray, int]:
   # Read the first linee manually
    with open(filepath, 'r') as file:
        num_elements = int(file.readline().strip())
    # Read the rest of the file (starting from the third line)
    element_file = pd.read_csv(filepath, sep=r'\s+', header=None, skiprows=1)
    # Convert to numpy array
    elements = element_file.to_numpy()
    # Add an extra row to accommodate 1-based indexing
    padded_elements = np.pad(elements, ((1,0),(0,0)), 'constant', constant_values=np.nan)

    return padded_elements, num_elements

element, num_elements = read_elements_file('test_cases_2d/elements')

print(f"number of elements: {num_elements}")
print("elements matrix:")
pprint(element)

number of elements: 6
elements matrix:
array([[nan, nan, nan, nan, nan],
       [ 1.,  1.,  2.,  1.,  1.],
       [ 2.,  1.,  3.,  1.,  1.],
       [ 3.,  1.,  4.,  1.,  1.],
       [ 4.,  2.,  3.,  1.,  1.],
       [ 5.,  2.,  4.,  1.,  1.],
       [ 6.,  3.,  4.,  1.,  1.]])


#### Compute Lengths ($L(i)$) and Direction cosines, $element\ cosines(i,j)$.

In [87]:
def compute_lengths_and_cosines(elements: np.ndarray, nodes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # get each element's 1st nodes
    node_1 = elements[1:, 1].astype(int)
    # get each element's 2nd nodes
    node_2 = elements[1:, 2].astype(int)

    # get coordinates of 1st nodes
    node_1_coordinates = nodes[node_1, 1:]
    # get coordinates of 2nd nodes
    node_2_coordinates = nodes[node_2, 1:]
    
    # compute change in coordinates for each positions.
    coordinate_displacements = node_2_coordinates - node_1_coordinates

    # for each element, compute length:
    element_lengths = np.linalg.norm(coordinate_displacements, axis=1)

    # for each element, compute element cosines
    element_cosines = coordinate_displacements / element_lengths.reshape(len(elements)-1, 1)

    # again, add an extra row to both lengths and cosine matrix to accomodate 1-based indexing:
    element_lengths = np.pad(element_lengths, (1,0), 'constant', constant_values=np.nan)
    element_cosines = np.pad(element_cosines, ((1,0), (1,0)), 'constant', constant_values=np.nan)

    return element_lengths, element_cosines

lengths, cosines = compute_lengths_and_cosines(element, node)

print("\nelement lenghts:")
pprint(lengths)
print("\nelement cosines")
pprint(cosines)



element lenghts:
array([       nan, 1.        , 1.        , 1.41421356, 1.41421356,
       1.        , 1.        ])

element cosines
array([[        nan,         nan,         nan],
       [        nan,  1.        ,  0.        ],
       [        nan,  0.        ,  1.        ],
       [        nan,  0.70710678,  0.70710678],
       [        nan, -0.70710678,  0.70710678],
       [        nan,  0.        ,  1.        ],
       [        nan,  1.        ,  0.        ]])


### "Forces" File

In [92]:
def read_forces_file(filepath: str = 'inputs/forces') -> Tuple[np.ndarray, int]:
   # Read the first linee manually
    with open(filepath, 'r') as file:
        num_force_BCs = int(file.readline().strip())
    # Read the rest of the file (starting from the third line)
    force_file = pd.read_csv(filepath, sep=r'\s+', header=None, skiprows=1)
    # Convert to numpy array
    forces = force_file.to_numpy()

    return forces, num_force_BCs

force_BCs, num_force_BCs = read_forces_file('test_cases_2d/forces')

print(f"\nNum of Force BCs: {num_force_BCs}")
print("Node #, DOF #, Value:")
pprint(force_BCs)


Num of Force BCs: 1
Node #, DOF #, Value:
array([[ 2.,  2., -1.]])


### "Displacements" File


In [93]:
def read_displacements_file(filepath: str = 'inputs/displacements') -> Tuple[np.ndarray, int]:
   # Read the first linee manually
    with open(filepath, 'r') as file:
        num_displacement_BCs = int(file.readline().strip())
    # Read the rest of the file (starting from the third line)
    displacement_file = pd.read_csv(filepath, sep=r'\s+', header=None, skiprows=1)
    # Convert to numpy array
    displacements = displacement_file.to_numpy()

    return displacements, num_displacement_BCs

displacement_BCs, num_displacement_BCs = read_forces_file('test_cases_2d/displacements')

print(f"\nNum of Displacement BCs: {num_displacement_BCs}")
print("Node #, DOF #, Value:")
pprint(displacement_BCs)


Num of Displacement BCs: 4
Node #, DOF #, Value:
array([[1., 1., 0.],
       [3., 1., 0.],
       [3., 2., 0.],
       [4., 1., 1.]])


### Stiffness and RHS Force Vector

In [99]:
# initilize an empty K matrix:
stiffness_matrix = np.zeros(shape=(total_num_dofs+1,total_num_dofs+1), dtype=float) # one more for 1 based indexing
# initialize an empty Force vector:
external_forces = np.zeros(shape=(total_num_dofs+1, 1), dtype=float)

print("\nK matrix")
print(stiffness_matrix)
print("RHS Force vector:")
print(external_forces)


K matrix
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
RHS Force vector:
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
